In [1]:
import face_recognition
import os
import cv2

In [2]:
KNOWN_FACES_DIR = 'C:/Users/johba/Desktop/Python/OPENCV/Known_faces'
UNKNOWN_FACES_DIR = 'C:/Users/johba/Desktop/Python/OPENCV/Unknown_faces'
TOLERANCE = 0.6
FRAME_THICKNESS = 3
FONT_THICKNESS = 2
MODEL = 'cnn'  # default: 'hog', other one can be 'cnn' - CUDA accelerated (if available) deep-learning pretrained model

known_faces = []
known_names = []

# Known faces
for name in os.listdir(KNOWN_FACES_DIR):
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        encoding = face_recognition.face_encodings(image)[0]
        known_faces.append(encoding)
        known_names.append(filename.split('.')[0])

# Unknown faces
for filename in os.listdir(UNKNOWN_FACES_DIR):
    image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')
   
    # This time we first grab face locations - we'll need them to draw boxes
    locations = face_recognition.face_locations(image)
    
    # Now since we know loctions, we can pass them to face_encodings as second argument
    # Without that it will search for faces once again slowing down whole process
    encodings = face_recognition.face_encodings(image, locations)
    
    # We passed our image through face_locations and face_encodings, so we can modify it
    # First we need to convert it from RGB to BGR as we are going to work with cv2
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    for face_encoding, face_location in zip(encodings, locations):
        # We use compare_faces (but might use face_distance as well)
        # Returns array of True/False values in order of passed known_faces
        results = face_recognition.compare_faces(known_faces, face_encoding)
        match = None
        
        if True in results:
            match = known_names[results.index(True)]
            print(f'Match found: {match}')
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])
            color = [0,255,0]            
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THICKNESS)
            
            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2]+22)
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)
            cv2.putText(image, match, (face_location[3]+10, face_location[2]+15), 
            cv2.FONT_HERSHEY_SIMPLEX, .5, (200,200,200), FONT_THICKNESS)

    cv2.imshow(filename, image)
    cv2.waitKey(0)

Match found: bill gates
Match found: bill gates
Match found: elon musk
Match found: elon musk
